In [40]:
import pandas as pd
import numpy as np
import random

from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

import re
from nltk import pos_tag
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer 

import seaborn as sns
import matplotlib.pyplot as plt


In [49]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dropout, Activation, Bidirectional
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

In [42]:
df = pd.read_csv('../../data/judge-1377884607_tweet_product_company.csv', encoding = 'latin1')
df.columns = ['text', 'product', 'emotion']
df = df[df['emotion'] != 'I can\'t tell']
df['product'].fillna('No Product', inplace = True)
df.dropna(inplace = True)

In [43]:
y = pd.get_dummies(df['emotion'])

In [44]:
y.head()

,Negative emotion,No emotion toward brand or product,Positive emotion
0,1,0,0
1,0,0,1
2,0,0,1
3,1,0,0
4,0,0,1


In [45]:
text_list = [str(x) for x in list(df['text'])]

corpus = []
for tweet in text_list:
    corpus.extend(tweet.split(' '))
    
len(set(corpus))

18955

tokenize tweets

In [46]:
text_list = [str(x) for x in list(df['text'])]
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(text_list)
list_tokenized_text = tokenizer.texts_to_sequences(text_list)
X_t = sequence.pad_sequences(list_tokenized_text, maxlen=100)

construct CNN with embedding layer

In [63]:
model = Sequential()

embedding_size = 128
model.add(Embedding(input_dim=len(set(corpus)), output_dim=embedding_size, input_length=100))
model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=30, activation='tanh'))
model.add(Dense(units=3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

fit model on tokenized data

In [64]:
model.fit(X_t, y, epochs=7, batch_size=32, validation_split=0.1)

Epoch 1/7
252/252 [==============================] - 9s 35ms/step - loss: 0.8114 - accuracy: 0.6124 - val_loss: 0.7427 - val_accuracy: 0.6387
Epoch 2/7
252/252 [==============================] - 8s 33ms/step - loss: 0.5957 - accuracy: 0.7457 - val_loss: 0.7099 - val_accuracy: 0.6633
Epoch 3/7
252/252 [==============================] - 8s 33ms/step - loss: 0.4050 - accuracy: 0.8401 - val_loss: 0.7726 - val_accuracy: 0.6465
Epoch 4/7
252/252 [==============================] - 9s 34ms/step - loss: 0.3069 - accuracy: 0.8834 - val_loss: 0.8015 - val_accuracy: 0.6622
Epoch 5/7
252/252 [==============================] - 8s 33ms/step - loss: 0.2518 - accuracy: 0.9021 - val_loss: 0.8266 - val_accuracy: 0.6723
Epoch 6/7
252/252 [==============================] - 9s 36ms/step - loss: 0.2202 - accuracy: 0.9126 - val_loss: 0.9046 - val_accuracy: 0.6667
Epoch 7/7
252/252 [==============================] - 9s 34ms/step - loss: 0.2083 - accuracy: 0.9159 - val_loss: 0.9278 - val_accuracy: 0.6566
